## 多分类器投票 ##

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
import numpy as np
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()
voting_clf=VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard')
X, y = make_moons()
from sklearn.cross_validation import train_test_split
#x为数据集的feature熟悉，y为label.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#voting_clf.fit(X_train,y_train)


D:\Anoconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#测试准确率
from sklearn.metrics import accuracy_score
for clf in(log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.9333333333333333
SVC 0.9
VotingClassifier 0.9


D:\Anoconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Bagging and Pasting #
有放回采样时baggin，没放回时pasting

In [3]:
#print(len(X_train))
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf=BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=70,bootstrap=True,n_jobs=-1)
bag_clf.fit(X_train,y_train)
y_pred=bag_clf.predict(X_test)
print(y_pred)
print(accuracy_score(y_test,y_pred))

[0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1]
0.9


# 包外估计 #
因为有30%的数据不会出现在训练集上，所以直接用oob来评价本身不需要在进行交叉验证和单独的验证机

In [4]:
bag_clf=BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,bootstrap=True,n_jobs=-1,oob_score=True)
bag_clf.fit(X_train,y_train)
print(bag_clf.oob_score_)

0.9428571428571428


In [5]:
y_pred=bag_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.9


## 随机森林 ##

In [7]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred=rnd_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))


0.9666666666666667


# 特征重要度 #
越靠近根部特征越重要

In [9]:
from sklearn.datasets import load_iris
iris=load_iris()
rnd_clf=RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris['data'],iris['target'])
for name,score in zip(iris['feature_names'],rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.1072391792533187
sepal width (cm) 0.022558383346090005
petal length (cm) 0.42940897177595744
petal width (cm) 0.44079346562463373


# adaboost #

In [13]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm='SAMME.R',
    learning_rate=0.5
)
ada_clf.fit(X_train,y_train)
y_pred=ada_clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9333333333333333


#  梯度提升 #

In [16]:
from sklearn.tree import DecisionTreeRegressor
tree_reg1=DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X_train,y_train)
#在第一个分类器的残差上拟合第二个分类器
y2=y_train-tree_reg1.predict(X_train)
tree_reg2=DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X_train,y2)
#在第二个分类器残差上拟合第三个分类器
y3=y2-tree_reg2.predict(X_train)
tree_reg3=DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X_train,y3)
y_pred=sum(tree.predict(X_test) for tree in(tree_reg1,tree_reg2,tree_reg3))

# 梯度提升决策树 #

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt=GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

为了找到树的最优数量，你可以使用早停技术。最简单使用这个技术的方法就是使用stagged_predict()：他在训练的每一个阶段返回一个迭代器，接下来代码用120个树训练了一个gbdt集成，然后在训练的每个阶段验证错误以找到树的最佳数量，最后使用gbdt树的最有数量训练另一个集成